In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.models import NHiTSModel
from darts.metrics import mse, rmse, r2_score, mae, smape
from darts.dataprocessing.transformers import Scaler
import torch
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import Callback, EarlyStopping
import optuna
import os
import json

# Visualization settings
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)
torch.set_float32_matmul_precision('medium')

In [2]:
target_columns = [
    'Temperature','Precipitation_accumulated','Humidity', 'Wind_Speed_kmh',
    'Soil_Moisture', 'Soil_Temperature', 'Wind_Dir_Sin', 'Wind_Dir_Cos'
]

DATA_FILE_PATH = "../data/ground_station_clean.csv"
df = pd.read_csv(DATA_FILE_PATH)

In [3]:
encoders = {
    "cyclic": {
        "past": ["month", "dayofyear", "day", "hour", "minute"],
        "future": ["month", "dayofyear", "day", "hour", "minute"]
    },
    "transformer": Scaler(),
    "datetime_attribute": {
        "past": ["year"],
        "future": ["year"]
    }
}


In [4]:
class PatchedPruningCallback(PyTorchLightningPruningCallback, Callback):
    pass

# Create directory to save iteration results
results_output_dir = "optuna_iteration_metrics"
os.makedirs(results_output_dir, exist_ok=True)

In [5]:
def objective(trial):
    input_chunk_length = trial.suggest_int("input_chunk_length", 24, 72)
    output_chunk_length = trial.suggest_int("output_chunk_length", 1, 24)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    batch_size = trial.suggest_int("batch_size", 64, 256)
    lr = trial.suggest_float("lr", 1e-6, 5e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)

    num_layers    = trial.suggest_int("num_layers", 1, 6)
    num_blocks    = trial.suggest_int("num_blocks", 1, 4)
    num_stacks    = trial.suggest_int("num_stacks", 1, 5)
    layer_widths    = trial.suggest_int("layer_widths", 32, 256)
    activation = trial.suggest_categorical("activation", ["ReLU", "GELU"])

    # Try different loss functions
    loss_functions = {
        'mse': torch.nn.MSELoss(),
        'mae': torch.nn.L1Loss(),
        'huber': torch.nn.HuberLoss(delta=1.0),
        'smoothl1': torch.nn.SmoothL1Loss()
    }
    loss_name = trial.suggest_categorical("loss_fn", list(loss_functions.keys()))

    # Try different optimizers
    optimizer_names = ['Adam', 'AdamW', 'RMSprop', 'SGD']
    optimizer_name = trial.suggest_categorical("optimizer", optimizer_names)
    optimizer_dict = {
        'Adam': torch.optim.Adam,
        'AdamW': torch.optim.AdamW,
        'RMSprop': torch.optim.RMSprop,
        'SGD': torch.optim.SGD
    }

    early_stopper = EarlyStopping("val_loss", min_delta=0.0005, patience=15, verbose=True)
    prunner = PatchedPruningCallback(trial, monitor="val_loss")
    pl_trainer_kwargs = {
        "accelerator": "auto",
        "callbacks": [early_stopper, prunner],
    }
    n = int(len(df) * 0.8)
    train_df_fold, val_df_fold = df.iloc[:n], df.iloc[n:]
    train_fold = TimeSeries.from_dataframe(train_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')
    val_fold = TimeSeries.from_dataframe(val_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')

    print(f"\nStarting Trial {trial.number}")
    print(f"Hyperparameters: {trial.params}")
    print("\nTraining the model...")
    print(f"Train set: {len(train_fold)} samples")
    print(f"Validation set: {len(val_fold)} samples")

    scaler = Scaler()
    scaler = scaler.fit(train_fold)
    train_scaled = scaler.transform(train_fold)
    val_scaled = scaler.transform(val_fold)

    _work_dir = "/home/eduardo/Documentos/Water-Cycle-Neural-Network/darts_logs/"
    _model_name = "model_optuna_temp"
    os.makedirs(_work_dir, exist_ok=True)

    # Additional optimizer kwargs based on optimizer type
    opt_kwargs = {"lr": lr, "weight_decay": weight_decay}
    if optimizer_name == 'SGD':
        opt_kwargs['momentum'] = 0.9

    model = NHiTSModel(
        model_name=_model_name,
        work_dir=_work_dir,
        input_chunk_length=input_chunk_length,
        output_chunk_length=output_chunk_length,
        activation=activation,
        num_layers=num_layers,
        num_blocks=num_blocks,
        num_stacks=num_stacks,
        layer_widths=layer_widths,
        n_epochs=30,
        batch_size=batch_size,
        dropout=dropout,
        add_encoders=encoders,
        pl_trainer_kwargs=pl_trainer_kwargs,
        loss_fn=loss_functions[loss_name],
        optimizer_cls=optimizer_dict[optimizer_name],
        lr_scheduler_cls=torch.optim.lr_scheduler.ReduceLROnPlateau,
        lr_scheduler_kwargs={"mode": "min", "factor": 0.3, "patience": 5, "min_lr": 1e-7},
        save_checkpoints=True,
        show_warnings=True,
        force_reset=True,
        optimizer_kwargs=opt_kwargs,
    )
    model.fit(
        series=train_scaled,
        val_series=val_scaled,
        verbose=False,
        dataloader_kwargs={"num_workers": 9},
    )

    try:
        loaded_model = NHiTSModel.load_from_checkpoint(model_name=_model_name, work_dir=_work_dir, best=True)
        print(f"Model loaded from checkpoint for trial {trial.number}")
    except FileNotFoundError:
        print(f"Checkpoint not found for {_model_name} in {_work_dir}. Using the in-memory trained model.")
        loaded_model = model

    forecasts = loaded_model.historical_forecasts(
        val_scaled,
        forecast_horizon=output_chunk_length,
        stride=1,
        retrain=False,
        verbose=False
    )

    forecasts_t = scaler.inverse_transform(forecasts)
    s = scaler.inverse_transform(val_scaled)
    metrics = {}
    print("Starting time series verification for consistency...")
    try:
        for target in target_columns:
            metrics[target] = {
                'MSE': mse(s[target], forecasts_t[target]),
                'RMSE': rmse(s[target], forecasts_t[target]),
                'MAE': mae(s[target], forecasts_t[target]),
                'R2': r2_score(s[target], forecasts_t[target]),
                'SMAPE': smape(s[target], forecasts_t[target]),
            }
        metrics_df = pd.DataFrame(metrics).T
        print("\nPerformance metrics:")
        print(metrics_df)
    except Exception as e:
        print(e)

    overall_mae_val = mae(val_scaled, forecasts)
    print(f"The MAE for this fold was {overall_mae_val}")

    trial_dict = {
        "trial_number": trial.number,
        "input_chunk_length": input_chunk_length,
        "output_chunk_length": output_chunk_length,
        "num_layers": num_layers,
        "num_blocks": num_blocks,
        "num_stacks": num_stacks,
        "layer_widths": layer_widths,
        "activation": activation,
        "dropout": dropout,
        "batch_size": batch_size,
        "lr": lr,
        "loss_function": loss_name,
        "optimizer": optimizer_name,
        "overall_mae_val": overall_mae_val,
        "metrics": metrics_df.to_dict(),
    }
    json_path = os.path.join(results_output_dir, f"trial_{trial.number}.json")
    with open(json_path, 'w') as f:
        json.dump(trial_dict, f, indent=4)
    print(f"Results of trial {trial.number} saved in {json_path}")
    return overall_mae_val if not np.isnan(overall_mae_val) else float("inf")


In [6]:
def print_callback(study, trial):
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")

In [7]:
study = optuna.create_study(direction="minimize")
num_hyperparams = 13
n_trials = 7 * num_hyperparams
study.optimize(objective, n_trials=n_trials, callbacks=[print_callback])

[I 2025-05-26 17:21:43,680] A new study created in memory with name: no-name-8cd29167-70aa-46ba-a1de-82deecc63e0b



Starting Trial 0
Hyperparameters: {'input_chunk_length': 57, 'output_chunk_length': 15, 'dropout': 0.3726858529535757, 'batch_size': 228, 'lr': 9.748201585551697e-05, 'weight_decay': 0.00044728672549721653, 'num_layers': 3, 'num_blocks': 1, 'num_stacks': 4, 'layer_widths': 131, 'activation': 'ReLU', 'loss_fn': 'mae', 'optimizer': 'SGD'}

Training the model...
Train set: 6647 samples
Validation set: 1662 samples


Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Metric val_loss improved. New best score: 0.510
Metric val_loss improved by 0.004 >= min_delta = 0.0005. New best score: 0.505
Metric val_loss improved by 0.004 >= min_delta = 0.0005. New best score: 0.501
Metric val_loss improved by 0.004 >= min_delta = 0.0005. New best score: 0.497
Metric val_loss improved by 0.004 >= min_delta = 0.0005. New best score: 0.493
Metric val_loss improved by 0.004 >= min_delta = 0.0005. New best score: 0.489
Metric val_loss improved by 0.004 >= min_delta = 0.0005. New best score: 0.486
Metric val_loss improved by 0.004 >= min_delta = 0.0005. New best score: 0.482
Metric val_loss improved by 0.004 >= min_delta = 0.0005. New best score: 0.478
Metric val_loss improv

Model loaded from checkpoint for trial 0


/home/eduardo/Documentos/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
[I 2025-05-26 17:22:12,458] Trial 0 finished with value: 0.35848179197382507 and parameters: {'input_chunk_length': 57, 'output_chunk_length': 15, 'dropout': 0.3726858529535757, 'batch_size': 228, 'lr': 9.748201585551697e-05, 'weight_decay': 0.00044728672549721653, 'num_layers': 3, 'num_blocks': 1, 'num_stacks': 4, 'layer_widths': 131, 'activation': 'ReLU', 'loss_fn': 'mae', 'optimizer': 'SGD'}. Best is trial 0 with value: 0.35848179197382507.
Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.
GPU available: True (cuda), used: True
TPU available: False, u

...

Current value: 0.02277054085767889, Current params: {'input_chunk_length': 35, 'output_chunk_length': 5, 'dropout': 0.1251552477475921, 'batch_size': 69, 'lr': 0.0004688683813510609, 'weight_decay': 2.1265746592638615e-06, 'num_layers': 1, 'num_blocks': 3, 'num_stacks': 4, 'layer_widths': 123, 'activation': 'ReLU', 'loss_fn': 'huber', 'optimizer': 'AdamW'}
Best value: 0.10596477232492901, Best params: {'input_chunk_length': 45, 'output_chunk_length': 1, 'dropout': 0.1300946376042601, 'batch_size': 104, 'lr': 0.00011494227662736073, 'weight_decay': 8.444384638688174e-05, 'num_layers': 1, 'num_blocks': 4, 'num_stacks': 3, 'layer_widths': 243, 'activation': 'GELU', 'loss_fn': 'huber', 'optimizer': 'AdamW'}

Starting Trial 89
Hyperparameters: {'input_chunk_length': 50, 'output_chunk_length': 1, 'dropout': 0.31869398458514314, 'batch_size': 94, 'lr': 0.0006249171047127408, 'weight_decay': 0.00019548992894327885, 'num_layers': 1, 'num_blocks': 4, 'num_stacks': 3, 'layer_widths': 226, 'activa

Metric val_loss improved. New best score: 0.029
[I 2025-05-26 18:56:04,093] Trial 89 pruned. Trial was pruned at epoch 1.
Specified future encoders in `add_encoders` at model creation but model does not accept future covariates. future encoders will be ignored.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Current value: 0.0263846387528189, Current params: {'input_chunk_length': 50, 'output_chunk_length': 1, 'dropout': 0.31869398458514314, 'batch_size': 94, 'lr': 0.0006249171047127408, 'weight_decay': 0.00019548992894327885, 'num_layers': 1, 'num_blocks': 4, 'num_stacks': 3, 'layer_widths': 226, 'activation': 'ReLU', 'loss_fn': 'huber', 'optimizer': 'RMSprop'}
Best value: 0.10596477232492901, Best params: {'input_chunk_length': 45, 'output_chunk_length': 1, 'dropout': 0.1300946376042601, 'batch_size': 104, 'lr': 0.00011494227662736073, 'weight_decay': 8.444384638688174e-05, 'num_layers': 1, 'num_blocks': 4, 'num_stacks': 3, 'layer_widths': 243, 'activation': 'GELU', 'loss_fn': 'huber', 'optimizer': 'AdamW'}

Starting Trial 90
Hyperparameters: {'input_chunk_length': 45, 'output_chunk_length': 3, 'dropout': 0.156471208337238, 'batch_size': 107, 'lr': 7.224017135386038e-05, 'weight_decay': 3.384214730445324e-05, 'num_layers': 2, 'num_blocks': 3, 'num_stacks': 2, 'layer_widths': 218, 'activa

[I 2025-05-26 18:56:11,030] Trial 90 pruned. Trial was pruned at epoch 0.


Current value: 0.04007860238876095, Current params: {'input_chunk_length': 45, 'output_chunk_length': 3, 'dropout': 0.156471208337238, 'batch_size': 107, 'lr': 7.224017135386038e-05, 'weight_decay': 3.384214730445324e-05, 'num_layers': 2, 'num_blocks': 3, 'num_stacks': 2, 'layer_widths': 218, 'activation': 'ReLU', 'loss_fn': 'smoothl1', 'optimizer': 'AdamW'}
Best value: 0.10596477232492901, Best params: {'input_chunk_length': 45, 'output_chunk_length': 1, 'dropout': 0.1300946376042601, 'batch_size': 104, 'lr': 0.00011494227662736073, 'weight_decay': 8.444384638688174e-05, 'num_layers': 1, 'num_blocks': 4, 'num_stacks': 3, 'layer_widths': 243, 'activation': 'GELU', 'loss_fn': 'huber', 'optimizer': 'AdamW'}


In [8]:
print(f"Best Value (Minimum): {study.best_value}")
print(f"Best Parameters: {study.best_params}")

Best Value (Minimum): 0.10596477232492901
Best Parameters: {'input_chunk_length': 45, 'output_chunk_length': 1, 'dropout': 0.1300946376042601, 'batch_size': 104, 'lr': 0.00011494227662736073, 'weight_decay': 8.444384638688174e-05, 'num_layers': 1, 'num_blocks': 4, 'num_stacks': 3, 'layer_widths': 243, 'activation': 'GELU', 'loss_fn': 'huber', 'optimizer': 'AdamW'}


In [9]:
best_dict = {
    "best_value": study.best_value,
    "best_params": study.best_params,
}
json_path = os.path.join(results_output_dir, "best_trial.json")
with open(json_path, 'w') as f:
    json.dump(best_dict, f, indent=4)
print(f"Best results saved in {json_path}")


Best results saved in optuna_iteration_metrics/best_trial.json
